In [0]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch import optim

In [0]:
batch_size = 64
num_of_channels = 3

In [4]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if train_on_gpu == True:
  print("CUDA is available!  Training on GPU ...")
else:
  print("CUDA is not available.  Training on CPU ...")

CUDA is available!  Training on GPU ...


In [0]:
transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [6]:
dataset = datasets.CIFAR10(
    'data',
    download = True,
    transform = transform
)


Extracting data/cifar-10-python.tar.gz to data


In [7]:
!ls

data  sample_data


In [8]:
dataset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [9]:
type(dataset)

torchvision.datasets.cifar.CIFAR10

In [0]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [0]:
class make_generator_model(nn.Module):
  def __init__(self):
    super(make_generator_model, self).__init__()
    
    self.main = nn.Sequential(
        
        nn.ConvTranspose2d(100, 512, kernel_size=4, stride=1, padding=0, bias=False), # inverse convolution since we are turning a vector into an image
        nn.BatchNorm2d(512),
        nn.ReLU(True),

        nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
        nn.BatchNorm2d(256),
        nn.ReLU(True),

        nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
        nn.BatchNorm2d(128),
        nn.ReLU(True),

        nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
        nn.BatchNorm2d(64),
        nn.ReLU(True),

        nn.ConvTranspose2d(64, num_of_channels, kernel_size=4, stride=2, padding=1, bias=False),
        nn.Tanh()
    )

  def forward(self, noise):
    return self.main(noise)

generator = make_generator_model()

In [0]:
class make_discriminator_model(nn.Module):
  def __init__(self):
    super(make_discriminator_model, self).__init__()

    self.main = nn.Sequential(
        
        nn.Conv2d(num_of_channels, 64, kernel_size=4, stride=2, padding=1, bias=False),
        nn.LeakyReLU(0.2),

        nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
        nn.BatchNorm2d(128),
        nn.LeakyReLU(0.2),

        nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
        nn.BatchNorm2d(256),
        nn.LeakyReLU(0.2),

        nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
        nn.BatchNorm2d(512),
        nn.LeakyReLU(0.2),

        nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=0, bias=False),
        nn.Sigmoid()
    )

  def forward(self, image):
    output = self.main(image)
    return output.view(-1)

discriminator = make_discriminator_model()

In [0]:
criterion = nn.BCELoss()

In [0]:
optimizer_generator = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_discriminator = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))